# Bringing your own GPU
This predictor demonstrates how to bring your own GPU for making predictions on Function. Function integrates with [Runhouse](https://www.run.house/) to send your predictor code to your own custom GPU instances.

> For startups, you'll find this especially useful if you have credits with the major cloud providers (e.g. AWS, Azure, GCP).

> For enteprirse workloads, we instead recommend bringing your own cluster or running Function on-prem. [Reach out](mailto:sales@fxn.ai) to learn more.

## Installing Runhouse
First, we'll need to install `rsync`:

In [ ]:
!apt-get install -y rsync

Now let's install Runhouse:

In [ ]:
# Install Runhouse
%pip install runhouse

To login to RH, we'll need a Runhouse token. Head over to [your Runhouse account](https://www.run.house/account) to generate your RH token, then create a Function environment variable with your Runhouse token:
```bash
# Open a terminal and run the following command:
fxn env create RH_TOKEN <Your Runhouse token>
```

## Adding a Function Guard
Before we write initialization code, we need to check whether a given block of code is executing in our Function predictor, or whether it is running on a Runhouse worker, which we create from the predictor:

## Logging into Runhouse

In [2]:
import runhouse as rh
from os import environ

rh.login(
    token=environ["RH_TOKEN"],
    download_secrets=True,
    download_config=True,
    upload_config=True,
    upload_secrets=True,
    interactive=False
)

⠹ Updating AWS catalog: aws/instance_quota_mapping.csv (every 7 hours)

INFO | 2023-09-25 12:52:58,271 | Saved secrets from Vault to local config files
INFO | 2023-09-25 12:52:58,286 | Found credentials in shared credentials file: ~/.aws/credentials
INFO | 2023-09-25 12:52:59,315 | Uploaded secrets for to Vault for: ['aws', 'sky']
INFO | 2023-09-25 12:52:59,316 | Successfully logged into Runhouse.


> Note: We've added a check to ensure that this code isn't executed on our Runhouse worker. It should only be executed in our Function predictor which creates the worker on Runhouse.

## Creating an On-Demand Cluster
Now, we can create a cluster on our AWS account:

In [ ]:
# Create an instance on AWS
rh.cluster(
    name="rh-a10x",
    instance_type="g5.2xlarge",
    provider="aws",
    autostop_mins=5,
    region="us-east-1"
).save()

## Implementing the Predictor
Let's create a predictor that uses StableDiffusion to generate an image from a text `prompt`. Unlike the standard `stable-diffusion` sample, here will be running the inference on a GPU in our own AWS account:

In [9]:
def stable_diffusion_generate (prompt: str):
    # According to RH, imports must be inside the function
    from diffusers import StableDiffusionPipeline
    from torch import float16
    # Create the SD pipeline
    pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-base", torch_dtype=float16, revision="fp16")
    # Send to the GPU
    pipeline.to("cuda")
    # Generate an image
    result = pipeline(prompt).images[0]
    # Return
    return result

INFO | 2023-07-19 08:22:45,046 | Attempting to load config for /olokobayusuf/rh-a10x from RNS.


The `stable_diffusion_generate` function will be sent to our GPU instance for inference. With this, we can now write our Function predictor:

In [ ]:
# Get our GPU cluster
gpu = rh.cluster(name="rh-a10x")

# Send the `stable_diffusion_generate` function to our GPU cluster using RH
stable_diffusion_generate = rh.function(fn=stable_diffusion_generate).to(
    gpu,
    env=["./", "torch --upgrade --extra-index-url https://download.pytorch.org/whl/cu117", "diffusers", "transformers"]
)

# Define the Function predictor
def predict (prompt: str):
    # Generate an image by sending to our custom cluster
    result = stable_diffusion_generate(prompt)
    # Return the result
    return result

## Creating the Predictor on Function
Because we're bringing our own GPU, we can create the predictor on Function and use CPU acceleration:
```bash
fxn create @username/byo-gpu bring-your-gpu.ipynb --overwrite
```

> Replace `username` with your Function username.